In [1]:
import sys
from pathlib import Path

# change to your NFF path
sys.path.insert(0, "..")
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../")

import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler

from nff.data import Dataset, split_train_validation_test, collate_dicts, to_tensor
from nff.train import Trainer, get_trainer, get_model, load_model, loss, hooks, metrics, evaluate

import argparse
from sigopt import Connection

from train import train
from forceconv import *

from MD17data import *

from forcepai import ForcePai
# from nff.nn.models import Painn

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("-logdir", type=str, default='./output')
parser.add_argument("-device", type=int, default=0)
parser.add_argument("-data", type=str, default='ethanol_dft')
params = vars(parser.parse_args([]))

DEVICE = params['device']
OUTDIR = '{}/{}/sandbox'.format(params['logdir'], 'test_ForcePai')

BATCH_SIZE = 10
lr = 1e-5
n_epochs = 100

In [3]:
data = get_MD17data(params['data'])
dataset = pack_MD17data(data, 10000)

In [4]:
train, val, test = split_train_validation_test(dataset, val_size=0.05, test_size=0.85)
train_loader = DataLoader(train, batch_size=BATCH_SIZE, collate_fn=collate_dicts)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, collate_fn=collate_dicts)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, collate_fn=collate_dicts)

In [5]:
modelparams = {"feat_dim": 128,
              "activation": "swish",
              "n_rbf": 20,
              "cutoff": 5.0,
              "num_conv": 3,
              "output_keys": ["energy"],
              "grad_keys": ["energy_grad"],
               # whether to sum outputs from all blocks in the model
               # or just the final output block. False in the original
               # implementation
              "skip_connection": False,
               # Whether the k parameters in the Bessel basis functions
               # are learnable. False originally
              "learnable_k": False,
               # dropout rate in the convolution layers, originally 0
               "conv_dropout": 0.0,
               # dropout rate in the readout layers, originally 0
               "readout_dropout": 0.0,
               # dictionary of means to add to each output key
               # (this is optional - if you don't supply it then
               # nothing will be added)
               # "means": {"energy": train.props['energy'].mean().item()},
               # dictionary of standard devations with which to 
               # multiply each output key
               # (this is optional - if you don't supply it then
               # nothing will be multiplied)
               # "stddevs": {"energy": train.props['energy'].std().item()}
              }
model = ForcePai(modelparams)
# model = get_model(modelparams, model_type="Painn")

In [6]:
loss_fn = loss.build_mse_loss(loss_coef={'energy_grad': 1})
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = Adam(trainable_params, lr=lr)
train_metrics = [
        metrics.MeanAbsoluteError('energy_grad')
    ]

In [7]:
train_hooks = [
    hooks.MaxEpochHook(n_epochs),
    hooks.CSVHook(
        OUTDIR,
        metrics=train_metrics,
    ),
    hooks.PrintingHook(
        OUTDIR,
        metrics=train_metrics,
        separator = ' | ',
        time_strf='%M:%S'
    ),
    hooks.ReduceLROnPlateauHook(
        optimizer=optimizer,
        patience=20,
        factor=0.5,
        min_lr=1e-7,
        window_length=1,
        stop_after_min=True
    )
]

T = Trainer(
    model_path=OUTDIR,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_loader=train_loader,
    validation_loader=val_loader,
    checkpoint_interval=1,
    hooks=train_hooks,
    mini_batches=1
)

In [8]:
T.train(device=DEVICE, n_epochs=1000)

 Time | Epoch | Learning rate | Train loss | Validation loss | MAE_energy_grad | GPU Memory (MB)


 99%|█████████▉| 99/100 [00:04<00:00, 23.88it/s]


01:30 |    11 |     1.000e-03 |    15.6437 |         10.8998 |          2.4804 |              33


 99%|█████████▉| 99/100 [00:03<00:00, 28.67it/s]


01:34 |    12 |     1.000e-03 |    10.1316 |          7.2117 |          2.0255 |              33


 99%|█████████▉| 99/100 [00:03<00:00, 26.03it/s]


01:38 |    13 |     1.000e-03 |     8.0491 |          6.3525 |          1.9071 |              33


 72%|███████▏  | 72/100 [00:02<00:01, 26.10it/s]


KeyboardInterrupt: 

In [98]:
from nff.utils.cuda import batch_to, batch_detach
data = None
for batch in train_loader:
    data = batch_to(batch, DEVICE)
    break

In [99]:
rotate = torch.Tensor([[2**.5/2, -2**.5/2, 0],
                       [2**.5/2, 2**.5/2, 0],
                       [0, 0, 1]]).to(DEVICE)
rotate

tensor([[ 0.7071, -0.7071,  0.0000],
        [ 0.7071,  0.7071,  0.0000],
        [ 0.0000,  0.0000,  1.0000]], device='cuda:0')

In [101]:
data['nxyz'][:, 1:4] @= rotate

In [100]:
results = batch_to(batch_detach(model(data)), DEVICE)

In [102]:
new_results = batch_to(batch_detach(model(data)), DEVICE)

In [116]:
(new_results['energy_grad'] - (results['energy_grad'] @ rotate)).mean()

tensor(8.0715e-08, device='cuda:0')

In [14]:
data["offsets"]

tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       device='cuda:0', size=(360, 3), nnz=0, layout=torch.sparse_coo)

In [112]:
results['energy_grad'] @ rotate

tensor([[ 6.0905e+01,  8.2145e+01, -5.9722e+01],
        [-1.0316e+02, -7.6270e+01, -3.6269e+01],
        [ 4.3973e+00,  3.2331e+01,  1.1046e+00],
        [ 1.8292e+00, -1.9102e+01,  5.2263e+01],
        [ 2.2242e+00, -3.6930e+01,  1.8547e+00],
        [ 3.4263e+01, -1.2126e+01, -9.8536e+00],
        [ 7.2883e+00,  1.7753e+01,  1.7333e+01],
        [-3.4666e+00,  3.5572e+01,  2.3576e+01],
        [-4.2809e+00, -2.3373e+01,  9.7131e+00],
        [-1.8791e+01, -2.1784e+01,  6.1356e+00],
        [ 3.9048e+01, -4.0592e+00, -8.6488e+00],
        [-2.2611e+01, -1.2323e+01, -1.3219e+01],
        [ 1.1123e+01, -5.8336e+00, -5.0618e+00],
        [ 1.9847e+01,  9.6305e+00, -1.6908e+01],
        [ 7.3554e+00, -3.6178e+00, -6.2092e-03],
        [-2.1006e+01,  1.0869e+00, -3.1800e+00],
        [-1.7629e+01,  3.1367e+01,  2.4362e+01],
        [ 2.6630e+00,  5.5336e+00,  1.6526e+01],
        [ 2.7739e+01, -1.7026e+01, -5.5337e+01],
        [ 1.2627e+01, -3.8149e+01, -3.0607e+01],
        [ 4.9671e+00

In [19]:
model.output_keys

['energy']